**Executed:** Mon Feb 20 17:44:35 2017

**Duration:** 4 seconds.

In [1]:
# default values of file paths, assuming quantum_fog is working dir
in_bif = 'examples_cbnets/earthquake.bif'
in_dot = 'examples_cbnets/earthquake.dot'
in_csv = 'learning/training_data_c/earthquake.csv'

In [2]:
# Cell inserted during automated execution.
in_bif = 'examples_cbnets/asia.bif'
in_dot = 'examples_cbnets/asia.dot'
in_csv = 'learning/training_data_c/tempo.csv'

**Parameter Learning Template**

# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Learned-Network-Parameters" data-toc-modified-id="Learned-Network-Parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Learned Network Parameters</a></div><div class="lev2 toc-item"><a href="#QFog" data-toc-modified-id="QFog-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>QFog</a></div><div class="lev2 toc-item"><a href="#bnlearn" data-toc-modified-id="bnlearn-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>bnlearn</a></div><div class="lev2 toc-item"><a href="#Summary-of-Running-Times" data-toc-modified-id="Summary-of-Running-Times-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Summary of Running Times</a></div>

In [3]:
import pandas as pd
import numpy as np
from graphviz import Source

import warnings
warnings.filterwarnings("ignore", module="rpy2")

import rpy2
%load_ext rpy2.ipython
%R library("bnlearn");
%R library("Rgraphviz");

In [4]:
# make this the absolute path of quantum-fog folder
qfog_path = '/home/jupyter/Notebooks/quantum-fog'

In [5]:
import os
import sys
if qfog_path != os.getcwd():
    os.chdir('../../')
    sys.path.insert(0,os.getcwd())
from learning.NetParamsLner import *

Read in_csv, create Pandas Dataframe with it, push dataframe into R

In [6]:
states_df = pd.read_csv(in_csv)
states_df.head()

,Smoke,Bronchitis,Lung-Cancer,Visited-Asian-TB-HRA,Tuberculosis,Either,X-Ray,Dyspnea
0,0,0,1,1,1,1,1,0
1,0,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1
3,0,0,1,1,1,1,1,0
4,1,1,1,1,1,1,1,1


In [7]:
states_df.tail()

,Smoke,Bronchitis,Lung-Cancer,Visited-Asian-TB-HRA,Tuberculosis,Either,X-Ray,Dyspnea
1995,0,0,1,1,1,1,1,0
1996,0,0,0,1,1,0,0,0
1997,0,1,1,1,1,1,0,1
1998,1,1,1,1,1,1,1,1
1999,0,1,1,1,1,1,1,1


In [8]:
%Rpush states_df
%R str(states_df)

'data.frame':	2000 obs. of  8 variables:
 $ Smoke               : int  0 0 1 0 1 0 0 0 1 1 ...
 $ Bronchitis          : int  0 1 1 0 1 0 1 0 0 1 ...
 $ Lung.Cancer         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Visited.Asian.TB.HRA: int  1 1 1 1 1 1 1 1 1 1 ...
 $ Tuberculosis        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Either              : int  1 1 1 1 1 1 1 1 1 1 ...
 $ X.Ray               : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Dyspnea             : int  0 1 1 0 1 0 1 0 0 1 ...


For bnlearn, data.frame columns cannot be int type, must be changed to factor.
data.frame is a list so can use lapply

In [9]:
%R states_df[] <- lapply(states_df, factor)
%R str(states_df)

'data.frame':	2000 obs. of  8 variables:
 $ Smoke               : Factor w/ 2 levels "0","1": 1 1 2 1 2 1 1 1 2 2 ...
 $ Bronchitis          : Factor w/ 2 levels "0","1": 1 2 2 1 2 1 2 1 1 2 ...
 $ Lung.Cancer         : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Visited.Asian.TB.HRA: Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Tuberculosis        : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Either              : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ X.Ray               : Factor w/ 2 levels "0","1": 2 2 2 2 2 2 2 2 2 2 ...
 $ Dyspnea             : Factor w/ 2 levels "0","1": 1 2 2 1 2 1 2 1 1 2 ...


# Learned Network Parameters

## QFog

In [10]:
# emp = empirical, learned
is_quantum = False
bnet = BayesNet.read_bif(in_bif, is_quantum)
bnet_emp = BayesNet.read_dot(in_dot)
vtx_to_states = bnet.get_vtx_to_state_names()
bnet_emp.import_nd_state_names(vtx_to_states)


lnr = NetParamsLner(is_quantum, bnet_emp, states_df)

In [11]:
%%capture qfog_params_time
%time lnr.learn_all_bnet_pots()

In [12]:
print(qfog_params_time)

CPU times: user 55.4 ms, sys: 0 ns, total: 55.4 ms
Wall time: 55.6 ms



In [13]:
lnr.compare_true_and_emp_pots(bnet, bnet_emp)


node= X-Ray
true:
['Either', 'X-Ray']
[[ 0.98  0.02]
 [ 0.05  0.95]]
empirical:
['Either', 'X-Ray']
[[ 0.98484848  0.01515152]
 [ 0.05513919  0.94486081]]

node= Either
true:
['Lung-Cancer', 'Tuberculosis', 'Either']
[[[ 1.  0.]
  [ 1.  0.]]

 [[ 1.  0.]
  [ 0.  1.]]]
empirical:
['Lung-Cancer', 'Tuberculosis', 'Either']
[[[ 1.  0.]
  [ 1.  0.]]

 [[ 1.  0.]
  [ 0.  1.]]]

node= Dyspnea
true:
['Bronchitis', 'Either', 'Dyspnea']
[[[ 0.9  0.1]
  [ 0.7  0.3]]

 [[ 0.8  0.2]
  [ 0.1  0.9]]]
empirical:
['Bronchitis', 'Either', 'Dyspnea']
[[[ 0.94736842  0.05263158]
  [ 0.69859515  0.30140485]]

 [[ 0.80357143  0.19642857]
  [ 0.07741935  0.92258065]]]

node= Bronchitis
true:
['Smoke', 'Bronchitis']
[[ 0.6  0.4]
 [ 0.3  0.7]]
empirical:
['Smoke', 'Bronchitis']
[[ 0.58811881  0.41188119]
 [ 0.26767677  0.73232323]]

node= Lung-Cancer
true:
['Smoke', 'Lung-Cancer']
[[ 0.1   0.9 ]
 [ 0.01  0.99]]
empirical:
['Smoke', 'Lung-Cancer']
[[ 0.1049505   0.8950495 ]
 [ 0.01111111  0.98888889]]

node= T

## bnlearn


In [14]:
%Rpush in_bif
%R bn.fit = read.bif(in_bif)
%R str(bn.fit)


Error in FUN(X[[i]], ...) : 
  dropped mismatched node is the parent of another node.


function (x, data, method = "mle", ..., debug = FALSE)  


In [15]:
%R bn = bn.net(bn.fit)
%R str(bn)


Error in check.fit(x) : x must be an object of class 'bn.fit'.



Error in str(bn) : object 'bn' not found


In [16]:
%%capture bnlearn_params_time
%time %R rfit = bn.fit(bn, data = states_df);

In [17]:
print(bnlearn_params_time)


Error in match(x, table, nomatch = 0L) : object 'bn' not found
CPU times: user 2.51 ms, sys: 124 us, total: 2.63 ms
Wall time: 2.75 ms



In [18]:
%R str(rfit)


Error in str(rfit) : object 'rfit' not found


In [19]:
%%R
j = 0
for( nd in nodes(bn.fit)){
    j = j + 1
    cat('----------------', nd[1], '\n')
    cat('true:\n')
    print(coef(bn.fit)[j])
    cat('empirical:\n')
    print(coef(rfit)[j])
}


Error in (function (classes, fdef, mtable)  : 
  unable to find an inherited method for function ‘nodes’ for signature ‘"function"’


## Summary of Running Times

In [20]:
print("QFog:\n", qfog_params_time)
print("bnlearn:\n", bnlearn_params_time)

QFog:
 CPU times: user 55.4 ms, sys: 0 ns, total: 55.4 ms
Wall time: 55.6 ms

bnlearn:
 
Error in match(x, table, nomatch = 0L) : object 'bn' not found
CPU times: user 2.51 ms, sys: 124 us, total: 2.63 ms
Wall time: 2.75 ms

